In [88]:
import requests
import json
import csv
import pandas as pd
from datetime import datetime
import geopandas as gpd
from dotenv import load_dotenv
load_dotenv()
import os
from google.cloud import storage

In [115]:
#1st dataset: historical case/death/hospitalization daily
#this is for visualizing the epidemic development curve
daily_summary = "https://raw.githubusercontent.com/nychealth/coronavirus-data/master/trends/data-by-day.csv"
daily_summary = pd.read_csv(daily_summary)
daily_summary.head()

,date_of_interest,CASE_COUNT,PROBABLE_CASE_COUNT,HOSPITALIZED_COUNT,DEATH_COUNT,PROBABLE_DEATH_COUNT,CASE_COUNT_7DAY_AVG,ALL_CASE_COUNT_7DAY_AVG,HOSP_COUNT_7DAY_AVG,DEATH_COUNT_7DAY_AVG,...,SI_HOSPITALIZED_COUNT,SI_DEATH_COUNT,SI_PROBABLE_DEATH_COUNT,SI_CASE_COUNT_7DAY_AVG,SI_PROBABLE_CASE_COUNT_7DAY_AVG,SI_ALL_CASE_COUNT_7DAY_AVG,SI_HOSPITALIZED_COUNT_7DAY_AVG,SI_DEATH_COUNT_7DAY_AVG,SI_ALL_DEATH_COUNT_7DAY_AVG,INCOMPLETE
0,02/29/2020,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,03/01/2020,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,03/02/2020,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,03/03/2020,1,0,7,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,03/04/2020,5,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [116]:
#2nd dataset: cases/death/hosp after vaccination by week (weekly breakthrough)
#the rate of confirmed/death/hospitalized people who have or haven't vaccined
breakthrough="https://raw.githubusercontent.com/nychealth/coronavirus-data/master/trends/weekly-breakthrough.csv"
breakthrough = pd.read_csv(breakthrough)
breakthrough.head()

,Week_of_diagnosis,vax_case_count,vax_case_rate,vax_hosp_count,vax_hosp_rate_adj,vax_death_count,vax_death_rate_adj,unvax_case_count,unvax_case_rate,unvax_hosp_count,unvax_hosp_rate_adj,unvax_death_count,unvax_death_rate_adj,citywide_case_rate,citywide_hosp_rate_adj,citywide_death_rate_adj,incomplete
0,01/23/2021,6,40.34,0,0.00,0,0.00,33170,480.41,2740,39.12,589,8.20,455.87,38.12,8.35,NaN
1,01/30/2021,22,52.16,2,7.74,2,6.40,31320,463.48,2599,38.70,550,7.94,431.98,36.95,7.74,NaN
2,02/06/2021,44,62.77,4,3.64,2,2.99,25106,379.53,2562,39.57,568,8.69,350.54,36.39,8.06,NaN
3,02/13/2021,66,55.18,3,3.02,1,1.16,24320,373.88,2269,36.58,461,7.37,338.55,32.79,6.53,NaN
4,02/20/2021,75,39.21,3,1.10,1,0.26,21589,336.89,2087,34.26,422,7.06,299.53,29.75,6.01,NaN


In [117]:
#3rd dataset: weekly cases/death/hosp rate by age or race
#weekly-case-rate-age
wk_case_age="https://raw.githubusercontent.com/nychealth/coronavirus-data/master/trends/weekly-case-rate-age.csv"
wk_case_age = pd.read_csv(wk_case_age)

#weekly-case-rate-race
wk_case_race="https://raw.githubusercontent.com/nychealth/coronavirus-data/master/trends/weekly-case-rate-race.csv"
wk_case_race = pd.read_csv(wk_case_race)

#merge case age and rate
wk_case=pd.merge(wk_case_age,wk_case_race)

#weekly-death-rate-age
wk_death_age="https://raw.githubusercontent.com/nychealth/coronavirus-data/master/trends/weekly-death-rate-age.csv"
wk_death_age= pd.read_csv(wk_death_age)

#weekly-death-rate-race
wk_death_race="https://raw.githubusercontent.com/nychealth/coronavirus-data/master/trends/weekly-death-rate-race.csv"
wk_death_race= pd.read_csv(wk_death_race)

wk_death=pd.merge(wk_death_age,wk_death_race)

wk_hosp_age="https://raw.githubusercontent.com/nychealth/coronavirus-data/master/trends/weekly-hosp-rate-age.csv"
wk_hosp_age= pd.read_csv(wk_hosp_age)

wk_hosp_race="https://raw.githubusercontent.com/nychealth/coronavirus-data/master/trends/weekly-hosp-rate-race.csv"
wk_hosp_race= pd.read_csv(wk_hosp_race)

wk_hosp=pd.merge(wk_hosp_age,wk_hosp_race)

#merge age and race data of case, hospital, death data by week
wk_case_death=pd.merge(wk_case,wk_death,how='outer',on=['week_ending'])

wk_all=pd.merge(wk_case_death,wk_hosp,how='outer',on=['week_ending'])

wk_all=wk_all.fillna(0)

wk_all.head()

,week_ending,age_all_ages_x,age_0_4_x,age_5_12_x,age_13_17_x,age_18_24_x,age_25_34_x,age_35_44_x,age_45_54_x,age_55_64_x,...,age_35_44,age_45_54,age_55_64,age_65_74,age_75up,all_race_ethnicity,Asian_Pacific_Islander,Black_African_American,Hispanic_Latino,White
0,02/29/2020,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,03/07/2020,0.29,0.00,0.00,0.00,0.00,0.00,0.00,0.68,0.00,...,0.00,0.00,0.00,0.00,0.00,0.13,0.00,0.00,0.00,0.00
2,03/14/2020,22.71,0.00,1.07,6.70,16.46,32.28,33.51,28.89,26.03,...,8.00,8.66,11.01,14.19,17.63,6.13,2.90,6.78,7.01,4.69
3,03/21/2020,221.93,14.13,17.66,57.30,148.86,259.22,321.93,315.25,304.33,...,43.10,65.17,90.80,119.78,159.54,47.19,20.30,55.30,57.43,30.38
4,03/28/2020,338.13,23.10,19.00,47.36,170.43,324.73,431.96,481.57,525.26,...,82.42,141.62,210.63,288.68,398.85,108.61,45.17,136.45,136.00,57.48


In [118]:
#4th datatset: this is used to map the distribution 
#last7days-by-modzcta
test_7d="https://raw.githubusercontent.com/nychealth/coronavirus-data/master/latest/last7days-by-modzcta.csv"
test_7d= pd.read_csv(test_7d)

#hosp_death_last28days-by-modzcta
hosp_death_28d="https://raw.githubusercontent.com/nychealth/coronavirus-data/master/latest/hosp_death_last28days-by-modzcta.csv"
hosp_death_28d= pd.read_csv(hosp_death_28d)

#first merge test_7d with hosp_death_28d
test_hosp_death=pd.merge(test_7d,hosp_death_28d,how='outer',on=['modzcta'])

#coverage-by-modzcta-allages: this only contains real-time vaccined data
vac_now_neigh="https://raw.githubusercontent.com/nychealth/covid-vaccine-data/main/people/coverage-by-modzcta-allages.csv"
vac_now_neigh= pd.read_csv(vac_now_neigh)
vac_now_neigh=vac_now_neigh.rename(columns={'MODZCTA': 'modzcta'})

#merge with test_hosp_death
data_map=pd.merge(test_hosp_death,vac_now_neigh,how='outer',on=['modzcta'])
data_map=data_map.rename(columns={'modzcta': 'MODZCTA'})

data_map.head()

,MODZCTA,modzcta_name_x,label_x,lat_x,lon_x,percentpositivity_7day,people_tested,people_positive,median_daily_test_rate,adequately_tested,...,BOROUGH,Label,AGE_GROUP,POP_DENOMINATOR,COUNT_PARTIALLY_CUMULATIVE,COUNT_FULLY_CUMULATIVE,COUNT_1PLUS_CUMULATIVE,PERC_PARTIALLY,PERC_FULLY,PERC_1PLUS
0,10001,Chelsea/NoMad/West Chelsea,"10001, 10118",40.750693,-73.997137,1.11,1799,20,1177.0,Yes,...,Manhattan,"10001, 10118",All ages,27613.09,7073,30039,37112,25.61,108.79,134.40
1,10002,Chinatown/Lower East Side,10002,40.715781,-73.986176,1.38,2889,40,678.4,Yes,...,Manhattan,10002,All ages,75322.71,5826,61957,67783,7.73,82.26,89.99
2,10003,East Village/Gramercy/Greenwich Village,10003,40.731825,-73.989164,2.05,4192,86,1526.6,Yes,...,Manhattan,10003,All ages,53977.81,4508,38881,43389,8.35,72.03,80.38
3,10004,Financial District,10004,40.703675,-74.013106,0.00,273,0,1480.4,Yes,...,Manhattan,10004,All ages,2972.12,654,3731,4385,22.00,125.53,147.54
4,10005,Financial District,10005,40.706092,-74.008861,2.94,476,14,1027.7,Yes,...,Manhattan,10005,All ages,8757.23,1041,7750,8791,11.89,88.50,100.39


In [127]:
#5th dataset: the historical vaccine data of NYC
vac_ago="https://health.data.ny.gov/api/views/duk7-xrni/rows.csv"
vac_ago= pd.read_csv(vac_ago)
vac_ago=vac_ago[vac_ago['Region']=="New York City"]
vac_ago=vac_ago.groupby('Report as of')
vac_ago=vac_ago.sum().reset_index()
vac_ago['Report as of']=pd.to_datetime(vac_ago1['Report as of'],format='%m/%d/%Y')
vac_ago=vac_ago.sort_values(by='Report as of')
vac_ago=vac_ago[:-1].iloc[1: , :]
vac_ago.head()

,Report as of,First Dose,Series Complete
326,2020-12-15,49,0
327,2020-12-16,944,1
328,2020-12-17,2980,6
329,2020-12-18,5360,15
330,2020-12-19,8583,30
...,...,...,...
320,2021-11-17,6431901,5746389
321,2021-11-18,6444135,5753807
322,2021-11-19,6457465,5761219
323,2021-11-20,6468953,5766001


In [110]:
#6th dataset: 
#NYC base map
NYC_map=gpd.read_file("https://raw.githubusercontent.com/nychealth/coronavirus-data/master/Geography-resources/MODZCTA_2010_WGS1984.geo.json")
NYC_map.head()

In [80]:
#Saving dataset to a file
daily_summary.to_csv('C:\\Users\\zheng\\Desktop\\MUSA 509 cloud comput\\509 final\\daily_summary.csv')

breakthrough.to_csv('C:\\Users\\zheng\\Desktop\\MUSA 509 cloud comput\\509 final\\breakthrough.csv')

wk_all.to_csv('C:\\Users\\zheng\\Desktop\\MUSA 509 cloud comput\\509 final\\wk_all.csv')

data_map.to_csv('C:\\Users\\zheng\\Desktop\\MUSA 509 cloud comput\\509 final\\data_map.csv')

vac_ago.to_csv('C:\\Users\\zheng\\Desktop\\MUSA 509 cloud comput\\509 final\\vac_ago.csv')

NYC_map = requests.get('https://raw.githubusercontent.com/nychealth/coronavirus-data/master/Geography-resources/MODZCTA_2010_WGS1984.geo.json') 
NYC_map = NYC_map.content
with open('C:\\Users\\zheng\\Desktop\\MUSA 509 cloud comput\\509 final\\NYC_map.geojson', 'wb') as f:
    f.write(NYC_map)

In [96]:
# Upload local file of data to Google Cloud Storage
bucket_name = os.environ['PIPELINE_DATA_BUCKET']  # <-- retrieve the bucket name from the environment
blob_name1 = f'daily_summary.csv'

storage_robot = storage.Client()
bucket = storage_robot.bucket(bucket_name)
blob1 = bucket.blob(blob_name1)
blob1.upload_from_filename(blob_name1)

blob_name2 = f'breakthrough.csv'
blob2 = bucket.blob(blob_name2)
blob2.upload_from_filename(blob_name2)

blob_name3 = f'wk_all.csv'
blob3 = bucket.blob(blob_name3)
blob3.upload_from_filename(blob_name3)

blob_name4 = f'data_map.csv'
blob4 = bucket.blob(blob_name4)
blob4.upload_from_filename(blob_name4)

blob_name5 = f'vac_ago.csv'
blob5 = bucket.blob(blob_name5)
blob5.upload_from_filename(blob_name5)

blob_name6 = f'NYC_map.geojson'
blob6 = bucket.blob(blob_name6)
blob6.upload_from_filename(blob_name6)